In [7]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
from keras import models
from keras import layers
import itertools
from sklearn.metrics import confusion_matrix
from keras import optimizers
from tensorflow.keras.optimizers import RMSprop
from PIL import Image
import pandas as pd
from keras.applications.vgg16 import VGG16
import os
import random

In [4]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

58892288/58889256 [==============================] - 143s 2us/step


In [9]:

    #Note
    for layer in vgg_conv.layers[:-5]:
        layer.trainable = False

    for layer in vgg_conv.layers:
        print(layer, layer.trainable)
        
    model = models.Sequential()    

    model.add(vgg_conv)
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(2, activation='softmax'))

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.RMSprop(lr=0.01),
                  metrics=['accuracy'])

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x0000020CA9D20B48> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020CA9EF8508> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020CA9EF8A88> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000020CA9A0B148> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020CAA363B88> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020CAA36EF08> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000020CAA371A08> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020CAA37C288> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020CAA389288> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000020CAA382748> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000020CAA374FC8> False
<tensorflow.python.

In [10]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              4719616   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 2050      
Total params: 19,436,354
Trainable params: 11,801,090
Non-trainable params: 7,635,264
_________________________________________________________________


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
TRAINING_DIR = 'Image_Dataset/Train'
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,target_size=(100,100),batch_size=5,class_mode='categorical')
VALIDATION_DIR = 'Image_Dataset/Test'
validation_datagen = ImageDataGenerator(rescale=1.0/255.0);
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,target_size=(100,100),batch_size=10,class_mode='categorical');

Found 1119 images belonging to 2 classes.
Found 280 images belonging to 2 classes.


In [16]:
history = model.fit(
      train_generator,
      steps_per_epoch=20,  # 2000 images = batch_size * steps
      epochs=10,
      validation_data=validation_generator,
      validation_steps=10)

Epoch 1/10
20/20 [==============================] - 54s 3s/step - loss: 7.6685 - accuracy: 0.5000 - val_loss: 7.9753 - val_accuracy: 0.4800
Epoch 2/10
20/20 [==============================] - 43s 2s/step - loss: 8.2820 - accuracy: 0.4600 - val_loss: 7.3618 - val_accuracy: 0.5200
Epoch 3/10
20/20 [==============================] - 46s 2s/step - loss: 7.3618 - accuracy: 0.5200 - val_loss: 7.5152 - val_accuracy: 0.5100
Epoch 4/10
20/20 [==============================] - 42s 2s/step - loss: 7.7460 - accuracy: 0.4949 - val_loss: 6.9017 - val_accuracy: 0.5500
Epoch 5/10
20/20 [==============================] - 42s 2s/step - loss: 7.3618 - accuracy: 0.5200 - val_loss: 6.5950 - val_accuracy: 0.5700
Epoch 6/10
20/20 [==============================] - 42s 2s/step - loss: 7.9753 - accuracy: 0.4800 - val_loss: 7.9753 - val_accuracy: 0.4800
Epoch 7/10
20/20 [==============================] - 43s 2s/step - loss: 7.8219 - accuracy: 0.4900 - val_loss: 7.9753 - val_accuracy: 0.4800
Epoch 8/10
20/20 [==